Copyright (c) 2018 - 2019 - [윤기태]

https://github.com/yoonkt200/python-data-analysis


# (가제) 파이썬 데이터 분석 - Code Examples

# 1.2 멕시코 프랜차이즈 Chipotle의 주문 데이터 분석하기

-----
-----

### 1. 데이터의 기초 정보 살펴보기

In [2]:
# -*- coding: utf-8 -*-

import pandas as pd

url = 'https://raw.githubusercontent.com/yoonkt200/python-data-analysis/master/data/chipotle.tsv'
chipo = pd.read_csv(url, sep = '\t') # read_csv 함수로 데이터를 Dataframe 형태로 불러옵니다.

In [10]:
print(chipo.shape)
print("------------------------------------")
print(chipo.info())

(4622, 5)
------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4622 entries, 0 to 4621
Data columns (total 5 columns):
order_id              4622 non-null int64
quantity              4622 non-null int64
item_name             4622 non-null object
choice_description    3376 non-null object
item_price            4622 non-null object
dtypes: int64(2), object(3)
memory usage: 180.6+ KB
None


In [12]:
chipo.head(10) # Dataframe에서 순서대로 10개의 row 데이터를 보여줍니다.

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98
5,3,1,Chicken Bowl,"[Fresh Tomato Salsa (Mild), [Rice, Cheese, Sou...",$10.98
6,3,1,Side of Chips,NaN,$1.69
7,4,1,Steak Burrito,"[Tomatillo Red Chili Salsa, [Fajita Vegetables...",$11.75
8,4,1,Steak Soft Tacos,"[Tomatillo Green Chili Salsa, [Pinto Beans, Ch...",$9.25
9,5,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Pinto...",$9.25


In [14]:
print(chipo.columns)
print("------------------------------------")
print(chipo.index)

Index(['order_id', 'quantity', 'item_name', 'choice_description',
       'item_price'],
      dtype='object')
------------------------------------
RangeIndex(start=0, stop=4622, step=1)
